In [1]:
import os
import requests
import time
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path

In [2]:
tickers = [
    'JPM', 
    'BAC', 
    'C', 
    'WFC', 
    'GS'
    ]

path = Path('.env.local')
load_dotenv(dotenv_path=path)

ALPHA_VANTAGE_API = os.getenv('DATA_API')

In [3]:
def fetch_stock_data(ticker, API_KEY):
    """Fetch daily stock data for a single ticker from Alpha Vantage"""
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={ticker}&apikey={ALPHA_VANTAGE_API}'
    response = requests.get(url)
    data = response.json()

    if "Error Message" in data:
        print(f"Error fetching {ticker}: {data['Error Message']}")
        return None
    if "Note" in data:
        print(f"API limit reached: {data['Note']}")
        return None
    
    time_series = data.get('Weekly Time Series', {})
    
    if not time_series:
        print(f"No time series data returned for {ticker}")
        return None
    
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    return df['4. close'].astype(float)

In [4]:
series_dict = {}

for ticker in tickers:
    series = fetch_stock_data(ticker, ALPHA_VANTAGE_API)
    if series is not None:
        series_dict[ticker] = series

    time.sleep(12)

No time series data returned for JPM
No time series data returned for BAC
No time series data returned for C
No time series data returned for WFC
No time series data returned for GS


In [5]:
bank_data = pd.DataFrame(series_dict)